In [ ]:

def stations_clusters_on_map(clustered_stations: pd.DataFrame):
    # Create a map centered at the mean coordinates of all stations
    center_lat = clustered_stations['latitude'].mean()
    center_lon = clustered_stations['longitude'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add markers for each station, colored by cluster
    colors = ['#%06x' % np.random.randint(0, 0xFFFFFF) for _ in range(clustered_stations["cluster"].nunique())]
    for _, row in clustered_stations.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=colors[int(row["cluster"])] if row["cluster"] != -1 else 'gray',
            fill=True,
            popup=f"Cluster {row["cluster"]}"
        ).add_to(m)

    """# Add markers for cluster centers
    for i, center in cluster_centers.iterrows():
        folium.Marker(
            location=[center['latitude'], center['longitude']],
            icon=folium.Icon(color='black', icon='info-sign'),
            popup=f'Cluster Center {i}'
        ).add_to(m)"""
    display(m)

stations_clusters_on_map(clustered_stations_dbscan)

In [ ]:
from matplotlib import colors as mcolors
from folium.plugins import HeatMap

# SUMMED DEMAND HEATMAP (cell 19)

# sum total demand per cluster across the dataset
cluster_total = df_demand.groupby('cluster')['demand'].sum().reset_index()

cluster_centers = clustered_stations_kmeans.groupby('cluster')[['latitude', 'longitude']].mean()
center_lat = cluster_centers['latitude'].mean()
center_lon = cluster_centers['longitude'].mean()

# merge with cluster centers (cluster_centers index corresponds to cluster id)
cluster_centers_idx = cluster_centers.reset_index().rename(columns={'index': 'cluster'})
cluster_total = cluster_total.merge(cluster_centers_idx, on='cluster', how='left').dropna(subset=['latitude','longitude'])

# normalize sizes and colors
max_d = cluster_total['demand'].max()
cluster_total['radius'] = (cluster_total['demand'] / max_d) * 40 + 5  # radius in pixels

cmap = plt.cm.viridis
norm = plt.Normalize(vmin=cluster_total['demand'].min(), vmax=max_d)

# create folium map centered on existing center
m_demand = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# add circle markers sized & colored by total demand
for _, r in cluster_total.iterrows():
    color = mcolors.to_hex(cmap(norm(r['demand'])))
    folium.CircleMarker(
        location=[r['latitude'], r['longitude']],
        radius=float(r['radius']),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Cluster {int(r['cluster'])} — Demand: {int(r['demand'])}"
    ).add_to(m_demand)

# add HeatMap layer (weights proportional to demand)
heat_data = [[r['latitude'], r['longitude'], r['demand']/max_d] for _, r in cluster_total.iterrows()]
HeatMap(heat_data, radius=25, max_zoom=13).add_to(m_demand)

m_demand

In [ ]:
# ================== Cluster demand metrics & visuals (no predictions) ==================
# Expects a DataFrame 'df' where:
#  - index is datetime
#  - columns: ['cluster','pickups','dropoffs','demand']

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) Wide matrices (time x cluster)
def build_cluster_wide(df: pd.DataFrame, value_col: str = "demand") -> pd.DataFrame:
    w = df.pivot_table(index=df.index, columns="cluster", values=value_col, aggfunc="sum", fill_value=0)
    # sort columns numerically if possible
    try:
        w = w.reindex(sorted(w.columns, key=lambda x: float(x)), axis=1)
    except Exception:
        pass
    return w

wide_demand  = build_cluster_wide(df_demand, "demand")
wide_pickups = build_cluster_wide(df_demand, "pickups")
wide_dropoff = build_cluster_wide(df_demand, "dropoffs")

# 2) Per-cluster metrics (level, variability, activity, share, seasonality proxy)
def _seasonal_lag(idx: pd.DatetimeIndex) -> int:
    if len(idx) > 1:
        step_h = np.median(np.diff(idx.values).astype("timedelta64[h]").astype(float))
        if step_h <= 1.5:  # hourly cadence
            return 24
    return 7

def cluster_demand_metrics(wide: pd.DataFrame) -> pd.DataFrame:
    lag = _seasonal_lag(wide.index)
    rows = []
    total_all = wide.sum().sum()
    for c in wide.columns:
        s = wide[c].astype(float)
        total = s.sum()
        rows.append({
            "cluster": c,
            "mean": s.mean(),
            "std": s.std(ddof=1),
            "total": total,
            "peak": s.max(),
            "active_periods": int((s > 0).sum()),
            "share_of_total_%": (total / (total_all + 1e-12)) * 100.0,
            f"autocorr_lag{lag}": float(s.autocorr(lag)) if len(s) > lag else np.nan
        })
    return pd.DataFrame(rows).sort_values("total", ascending=False).reset_index(drop=True)

metrics = cluster_demand_metrics(wide_demand)
display(metrics.head(15))

# 3) Correlation heatmap (how clusters co-move over time)
def plot_cluster_corr_heatmap(wide: pd.DataFrame, method: str = "pearson", title_suffix: str = ""):
    corr = wide.corr(method=method)
    fig = plt.figure(figsize=(8, 7)); ax = plt.gca()
    im = ax.imshow(corr.values, aspect="auto")
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04); cbar.set_label(f"{method.title()} correlation")
    labels = [str(int(c)) if isinstance(c, (int, np.integer, float)) and float(c).is_integer() else str(c)
              for c in corr.columns]
    ax.set_xticks(range(len(labels))); ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=90); ax.set_yticklabels(labels)
    ax.set_title(f"Cluster demand correlation ({method}) {title_suffix}")
    ax.set_xlabel("Cluster"); ax.set_ylabel("Cluster")
    plt.tight_layout()
    return corr

corr_pearson = plot_cluster_corr_heatmap(wide_demand, method="pearson")
plt.show()

# 4) Hour-of-day profile heatmap (avg demand by hour & cluster)
def plot_hourly_profile_heatmap(df: pd.DataFrame, value_col: str = "demand"):
    g = df.copy()
    g["hour"] = g.index.hour
    prof = g.pivot_table(index="hour", columns="cluster", values=value_col, aggfunc="mean", fill_value=0)
    # keep columns order similar to wide
    try:
        prof = prof.reindex(sorted(prof.columns, key=lambda x: float(x)), axis=1)
    except Exception:
        pass
    fig = plt.figure(figsize=(10, 5)); ax = plt.gca()
    im = ax.imshow(prof.values, aspect="auto", origin="lower")
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04); cbar.set_label(f"Avg {value_col}")
    ax.set_yticks(range(0, 24)); ax.set_yticklabels(range(0, 24))
    ax.set_xlabel("Cluster"); ax.set_ylabel("Hour of day")
    ax.set_xticks(range(prof.shape[1]))
    ax.set_xticklabels([str(int(c)) if isinstance(c,(int,np.integer,float)) and float(c).is_integer() else str(c)
                        for c in prof.columns], rotation=90)
    ax.set_title(f"Hourly average {value_col} by cluster")
    plt.tight_layout()

plot_hourly_profile_heatmap(df_demand, value_col="demand")
plt.show()

# 5) Share-of-total bar chart (who contributes most to system demand)
def plot_cluster_total_share(metrics_df: pd.DataFrame, title_suffix=""):
    m = metrics_df.sort_values("total", ascending=False)
    x = np.arange(len(m))
    plt.figure(figsize=(10, 4))
    plt.bar(x, m["total"].values)
    plt.xticks(x, [str(int(c)) if isinstance(c,(int,np.integer,float)) and float(c).is_integer() else str(c)
                   for c in m["cluster"]], rotation=0)
    plt.ylabel("Total demand"); plt.xlabel("Cluster")
    plt.title(f"Total demand by cluster {title_suffix}")
    plt.tight_layout()

plot_cluster_total_share(metrics)
plt.show()

# 6) Optional: top co-moving cluster pairs (based on Pearson corr on demand)
def top_corr_pairs(corr: pd.DataFrame, k: int = 10) -> pd.DataFrame:
    c = corr.copy(); np.fill_diagonal(c.values, np.nan)
    pairs = []
    cols = c.columns.to_list()
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            pairs.append((cols[i], cols[j], c.iloc[i, j]))
    out = pd.DataFrame(pairs, columns=["cluster_i","cluster_j","corr"]).sort_values("corr", ascending=False)
    return out.head(k)

display(top_corr_pairs(corr_pearson, k=10))


In [ ]:
# === Pickup (cluster i) vs Dropoff (cluster j) correlation ===
# Expects df with index=datetime and columns: ['cluster','pickups','dropoffs','demand']

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def _build_wide(df: pd.DataFrame, value_col: str) -> pd.DataFrame:
    w = df.pivot_table(index=df.index, columns="cluster", values=value_col, aggfunc="sum", fill_value=0)
    try:
        w = w.reindex(sorted(w.columns, key=lambda x: float(x)), axis=1)
    except Exception:
        pass
    return w

def pickup_dropoff_corr_matrix(df: pd.DataFrame, lag_hours: int = 0, method: str = "pearson") -> pd.DataFrame:
    """
    Correlate pickups at cluster i with dropoffs at cluster j.
    lag_hours>0 means dropoffs occur later; we align pickups(t) with dropoffs(t+lag).
    """
    P = _build_wide(df, "pickups")
    D = _build_wide(df, "dropoffs")

    if lag_hours != 0:
        # align pickups(t) with dropoffs(t+lag)
        D = D.shift(-lag_hours)

    # align time index
    common = P.index.intersection(D.index)
    P, D = P.loc[common], D.loc[common]

    # drop constant columns (std=0) to avoid NaNs-only columns
    P = P.loc[:, P.std(ddof=0) > 0]
    D = D.loc[:, D.std(ddof=0) > 0]

    # compute matrix: rows = pickup clusters, cols = dropoff clusters
    mat = pd.DataFrame(index=P.columns, columns=D.columns, dtype=float)
    for dcol in D.columns:
        mat[dcol] = P.corrwith(D[dcol], axis=0, method=method)
    return mat

def plot_pickup_dropoff_corr_heatmap(corr: pd.DataFrame, title_suffix: str = ""):
    fig = plt.figure(figsize=(9, 7)); ax = plt.gca()
    im = ax.imshow(corr.values, aspect="auto")
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Correlation")

    xt = [str(int(c)) if isinstance(c,(int,np.integer,float)) and float(c).is_integer() else str(c) for c in corr.columns]
    yt = [str(int(r)) if isinstance(r,(int,np.integer,float)) and float(r).is_integer() else str(r) for r in corr.index]
    ax.set_xticks(range(len(xt))); ax.set_yticks(range(len(yt)))
    ax.set_xticklabels(xt, rotation=90); ax.set_yticklabels(yt)
    ax.set_xlabel("Dropoff cluster (j)"); ax.set_ylabel("Pickup cluster (i)")
    ax.set_title(f"Corr[pickups_i, dropoffs_j] {title_suffix}")
    plt.tight_layout()

# ==== Run (examples) ====
# corr_0h: same-hour correlation
corr_0h = pickup_dropoff_corr_matrix(df_demand, lag_hours=0, method="pearson")
plot_pickup_dropoff_corr_heatmap(corr_0h, title_suffix="(lag=0h)")
plt.show()

# corr_2h: pickups(t) vs dropoffs(t+2h) correlation (helpful if effects are delayed)
corr_2h = pickup_dropoff_corr_matrix(df_demand, lag_hours=2, method="pearson")
plot_pickup_dropoff_corr_heatmap(corr_2h, title_suffix="(lag=2h)")
plt.show()



Here’s a tight, battle-tested strategy for Task 2 that fits your setup (clusters already fixed, hourly data, Jan–Oct train / Nov–Dec test).

1) What to predict & how to structure the data

Targets (per cluster c):

pickups_c(t) for each hour of Day+1 (24 steps)

dropoffs_c(t) for each hour of Day+1 (24 steps)

Data you should keep (both!)

Cluster-level time series: datetime, cluster, pickups, dropoffs, demand

OD table (you already have): start_cluster, end_cluster, start_datetime_hour, stop_datetime_hour

Rationale: cluster series are perfect for univariate/seasonal models; OD lets you model dropoffs as a function of earlier pickups elsewhere (travel). Keep both paths available.

2) Baselines (must-have yardsticks)

Seasonal naïve (24h): 
𝑦
^
𝑡
+
ℎ
=
𝑦
𝑡
+
ℎ
−
24
y
^
	​

t+h
	​

=y
t+h−24
	​


Weekly seasonal (168h): 
𝑦
^
𝑡
+
ℎ
=
𝑦
𝑡
+
ℎ
−
168
y
^
	​

t+h
	​

=y
t+h−168
	​


ETS (Holt-Winters) per cluster for quick strong baseline.

Use them for both pickups & dropoffs; never skip baselines.

3) Pickups model (per cluster)

Pickups are largely cluster-local + strong daily/weekly seasonality. Two robust options:

A) SARIMA / ExponentialSmoothing (per cluster)

Train per cluster on Jan–Oct.

24-step direct forecast at midnight (multi-step from the fitted model).

Good when signal is seasonal/stationary; quick & dependable.

B) Gradient-boosted trees (XGBoost/LightGBM) with lags

Features: hour, dow, month, pickups_lag{1,24,168}, rolling{24,168}, optional: dropoffs_lag{…}.

Direct multi-output: either recursive (predict hour h then feed forward) or 24 small “direct” models.

Strong when seasonality + nonlinear effects; fast to iterate.

👉 Practical plan: train both per cluster; keep the best per-cluster (MAE/WAPE on Nov–Dec). In many bike datasets, ETS/SARIMA wins for quieter clusters, XGB wins for busier/noisier ones.

4) Dropoffs model (per cluster): two levels

Dropoffs at cluster d are the result of earlier pickups at origins o with travel time. Use the OD table.

Level 1 — OD kernel (recommended):
Learn a transition kernel 
𝐾
𝑜
→
𝑑
(
𝜏
)
K
o→d
	​

(τ) = P(dropoff at d occurs 
𝜏
τ hours after a pickup at o) from Jan–Oct:

Build histograms of τ = stop_hour − start_hour per (o,d), normalize across d,τ.

(Optionally) condition on hour-of-day / weekday for better realism.

Predicting Day+1 dropoffs: convolve your predicted pickups with 
𝐾
K:

𝑑
𝑟
𝑜
𝑝
𝑜
𝑓
𝑓
𝑠
^
𝑑
(
𝑡
)
=
∑
𝑜
∑
𝜏
𝑝
𝑖
𝑐
𝑘
𝑢
𝑝
𝑠
^
𝑜
(
𝑡
−
𝜏
)
 
𝐾
𝑜
→
𝑑
(
𝜏
)
dropoffs
	​

d
	​

(t)=
o
∑
	​

τ
∑
	​

pickups
	​

o
	​

(t−τ)K
o→d
	​

(τ)

Pros: causal, interpretable, uses OD you already have.
Calibrate smoothing: shrink tiny OD pairs, cap τ (e.g., 0–6h window).

Level 2 — Local refinement (optional):
Per-cluster SARIMAX with exogenous terms = lagged local pickups and/or inbound pickups from top origins (from OD).
This soaks up residual seasonality the kernel misses.

Fallback if you skip OD: per-cluster SARIMAX with exogenous = own pickups_lag{1,2,…}, or a VAR( [pickups, dropoffs] ) per cluster. Works, but gives up spatial causality.

5) Train / validation protocol (strictly time-aware)

Train: 2018-01-01 … 2018-10-31 23:00

Test: 2018-11-01 … 2018-12-31

For every test day D, fit/retain your model on data ≤ D-1 23:00, then predict D’s 24 hours.

Keep same split for all clusters, no shuffling.

6) Features to prepare (minimal → strong)

Always: hour, day_of_week, month.

Lags & rolls (per target): lag{1,24,168}, rolling mean/std {24,168}.

For OD kernel: counts of flows per (o,d,τ); store as a sparse mapping or small arrays per (o,d).

Optional extras (if you later add them): weather (temp, precip), holiday flags.

Schema answers to your question:

Keep both:

time–cluster–pickups/dropoffs (for per-cluster models/metrics) and

time–start_cluster (pickups) plus time–end_cluster (dropoffs) via your OD table (for the kernel).
They serve different modeling steps and you need both for the best dropoff predictions.

7) Metrics to decide “what wins”

Report per cluster and overall (Nov–Dec):

MAE, RMSE, WAPE, sMAPE, MASE(24), Directional Accuracy.

Hour-of-day MAE profiles (where it struggles).

For dropoffs, also OD distribution fidelity (e.g., Jensen–Shannon between true vs predicted destination share from each origin).

8) Minimal implementation sketch (pseudo-code)

Pickups per cluster

for cluster c:
    fit SARIMA and XGB on Jan–Oct
    for each test day D:
        ŷ_pu_c[D’s 24h] = model.predict_24h(history ≤ D-1 23:00)
    choose best model per-cluster by WAPE


OD kernel for dropoffs

# TRAIN (Jan–Oct)
for each (o,d):
    histogram τ = stop_hour - start_hour  within τ ∈ [0..6] (example)
    normalize over d,τ with smoothing -> K[o,d,τ]

# PREDICT Day D
for each destination d and hour t ∈ D:
    ŷ_do_d(t) = sum_o sum_τ ŷ_pu_o(t-τ) * K[o,d,τ]


(Optional) Local SARIMAX refinement on residuals.

9) Why this split of responsibilities works

Pickups are “supply leaving” decisions localized in space–time → per-cluster seasonal models excel.

Dropoffs are consequences of earlier pickups elsewhere with a travel-time kernel → the OD convolution is the most faithful and usually beats standalone univariate models for dropoffs.

If you want, I can drop in ready-to-paste notebook cells for:

building the OD kernel 
𝐾
𝑜
→
𝑑
(
𝜏
)
K
o→d
	​

(τ) from your table,

the day-ahead convolution using your existing pickups forecasts, and

the evaluation tables/plots (WAPE per cluster, OD heatmaps of error).